# Mạng CNN cho bộ dữ liệu cifar10
Bộ tham số được train trên Google Colab để sử dụng GPU miễn phí. `colab_log.ipynb` có chứa hiện trạng notebook khi vừa train xong.

Trước hết cần đặt các biến setting:

In [1]:
GOOGLE_COLAB = False
PROJ_PATH = 'drive/cifar10_cnn'
MODEL = 'model_1525796201' # tên file weight, để None để train

`model_1525796201` được train với 75 epoch, đạt accuracy 0.8307 trên bộ dữ liệu test. Nhìn log thì vẫn chưa hội tụ, nếu tăng số epoch có thể đạt được accuracy cao hơn.

Nếu chạy trên máy Colab, cài Keras và mount Google Drive để lưu tham số train được:

In [2]:
if GOOGLE_COLAB:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
    !apt-get update -qq 2>&1 > /dev/null
    !apt-get -y install -qq google-drive-ocamlfuse fuse
    !pip install -q keras
    
    from google.colab import auth
    auth.authenticate_user()
    from oauth2client.client import GoogleCredentials
    creds = GoogleCredentials.get_application_default()
    import getpass
    !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
    vcode = getpass.getpass()
    !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
    
    !mkdir -p drive
    !google-drive-ocamlfuse -o nonempty drive

!mkdir -p drive/cifar10_cnn

Import Keras, load và format dữ liệu:

In [3]:
from keras.callbacks import EarlyStopping
from keras.datasets import cifar10
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adagrad
from keras.utils import to_categorical

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

/home/lazyc97/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(50000, 32, 32, 3) (10000, 32, 32, 3)
(50000, 10) (10000, 10)


Bước tiếp theo là tạo model, cấu trúc đề xuất:
- 2 lớp Convolution 3x3 với 64 filter, sử dụng hàm relu làm hàm kích hoạt.
- 1 lớp max pooling 2x2.
- 1 lớp dropout để chống overfit, loại bỏ 50% nút.
- Thêm 4 lớp tương tự trên nhưng thay kích thước kernel 2 lớp Convolution thành 5x5.
- 1 lớp densely-connected 512 loại.
- 1 lớp dropout 50%.
- 1 lớp fully-connected để đưa ra kết quả cuối cùng.

Hàm loss: `categorical_crossentropy`, bên trong sử dụng hàm softmax.  
Optimizer: `adagrad`, sử dụng thuật tối ưu này không cần tune tốc độ học.

Cài đặt:

In [4]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])

In [5]:
if MODEL is None:
    model.fit(
      X_train, y_train,
      batch_size=100,
      epochs=75,
      validation_data=(X_test, y_test),
      shuffle=True,
    )
else:
    model.load_weights(PROJ_PATH + '/' + MODEL)

In [7]:
print('acc = ', model.evaluate(X_test, y_test)[1])

10000/10000 [==============================] - 21s 2ms/step
acc =  0.8307


In [8]:
if MODEL is None:
    from time import time
    model.save(PROJ_PATH + '/model_' + str(int(time())))